In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Paramètres de la pipeline

In [110]:
code_departement = "093"
logs_level = "WARNING"

# Imports & setup

In [ ]:
import duckdb
import geopandas as gpd
import pandas as pd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DEFAULT_CRS, ALGORITHME_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.bd_pci import extract_bd_pci
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.sources.protected_buildings import extract_protected_buildings, get_areas_with_protected_buildings
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by_etablishment
from potentiel_solaire.geojson_files_creator import export_to_geojson
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)
pd.options.display.max_columns = None

# Extraction des données sources

### Etablissements scolaires

In [112]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

Annuaire des établissements scolaires extrait ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\fr-en-annuaire-education.geojson


### BD TOPO

In [113]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

BD TOPO extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15\BDTOPO\1_DONNEES_LIVRAISON_2024-12-00134\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg


### BD PCI

In [ ]:
bd_pci_path = extract_bd_pci(code_departement=code_departement)
print(f"BD PCI extraite ici: {bd_pci_path}")

### BD Protected Buildings

In [116]:
bd_protected_buildings_path = extract_protected_buildings()
print(f"BD des bâtiments protégés extraite ici: {bd_protected_buildings_path}")

BD des bâtiments protégés extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data/liste_immeubles_proteges.geojson


# Filtre des données sur le périmètre du calcul

### Etablissements scolaires


In [117]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


### Zone d'intérêt géographique

In [118]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

### Zones d'éducations

In [119]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_n

Nb de zones d'éducations:  1088


### Bâtiments

In [120]:
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_n

Nb de batiments:  351576


### Zones avec des bâtiments protégés

In [121]:
areas_with_protected_buildings = get_areas_with_protected_buildings(
    bd_protected_buildings_path=bd_protected_buildings_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)

# Détermination des bâtiments scolaires

In [122]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Nb de batiments scolaires:  3731


# Calcul des attributs utiles pour le potentiel solaire

In [ ]:
# TODO: v0 seulement à ce stade
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
    areas_with_protected_buildings=areas_with_protected_buildings,
    geom_of_interest=geom_of_interest
)

# Dump des donnees pour analyses

In [124]:
layers = ["schools_establishments", "educational_zones", "schools_buildings", "solar_potential_of_schools_buildings"]
gdfs = [schools_establishments, educational_zones, schools_buildings, solar_potential_of_schools_buildings]

for layer, gdf in zip(layers, gdfs):
    output_gpkg = DATA_FOLDER / f"{code_departement}_pipeline_results.gpkg"
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG")

# Checks sur la qualité des données & calculs

In [125]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

Nb d'établissements scolaires avec des batiments: 445 (39%)


In [126]:
nb_buildings_protected = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.protection].shape[0]
print(f"Nb de batiments protégés {nb_buildings_protected} ({100*nb_buildings_protected/nb_schools_buildings:.2f}%)")

Nb de batiments protégés 1093 (29.30%)


# Sauvegarde des calculs en Base de Données et en fichiers GeoJson

### Evaluation du Potentiel solaire par Etablissement

In [127]:
results_by_school = aggregate_solar_potential_by_etablishment(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings
)

results_by_school.head()

,identifiant_de_l_etablissement,surface_utile,rayonnement_solaire,potentiel_solaire,protection
0,0930043S,0.000000,0.000000,0.000000,False
1,0930089S,3245.297871,1143.404907,371066.369436,True
2,0930100D,2069.812266,1143.329590,236647.053457,False
3,0930116W,0.000000,0.000000,0.000000,False
4,0930117X,4907.794034,1141.873413,560407.952443,True


### Sauvegarde des calculs en Base de Données

In [141]:
conn = duckdb.connect('./../database/potentiel_solaire.duckdb')

# Load Spatial extension

conn.execute("""
    INSTALL spatial;
    LOAD spatial;
             """)

conn.commit();

#### Mise à jour des données des Etablissements du Département en Base de Données

In [ ]:
conn.register("results_by_school", results_by_school)

update_query = """
UPDATE etablissements
SET 
    surface_utile = results_by_school.surface_utile,
    rayonnement_solaire = results_by_school.rayonnement_solaire,
    potentiel_solaire = results_by_school.potentiel_solaire,
    protection = results_by_school.protection
FROM
    results_by_school
WHERE
    etablissements.identifiant_de_l_etablissement = results_by_school.identifiant_de_l_etablissement
"""

conn.execute(update_query)

query = f"""
SELECT
    *,
    ST_AsGeoJSON(geo_point) as geometry
FROM
    etablissements
WHERE
code_departement = '{code_departement}'
"""

stats_schools = conn.query(query).df()

assert stats_schools.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des établissements"

stats_schools.head(3)

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,libelle_nature,code_commune,nom_commune,code_departement,libelle_departement,code_region,libelle_region,surface_utile,rayonnement_solaire,potentiel_solaire,protection,geo_point,geometry
0,0930836D,Ecole élémentaire Ambourget 2,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93005,Aulnay-sous-Bois,093,Seine-Saint-Denis,11,Ile-de-France,2596,1145,297303,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.5101234546026..."
1,0930155N,Ecole élémentaire Romain Rolland,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93048,Montreuil,093,Seine-Saint-Denis,11,Ile-de-France,0,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.4757109501393..."
2,0930158S,Ecole élémentaire Jean Rostand,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93072,Stains,093,Seine-Saint-Denis,11,Ile-de-France,1149,1142,131260,True,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.3869213324153..."


#### Mise à jour des données des Communes du Département en Base de Données

In [ ]:
update_query = f"""
UPDATE communes
SET 
    surface_utile = agg.surface_utile,
    potentiel_solaire = agg.potentiel_solaire,
    count_etablissements = agg.count_etablissements,
    count_etablissements_proteges = agg.count_etablissements_proteges
FROM (
    SELECT 
         code_commune,
         SUM(surface_utile) AS surface_utile,
         SUM(potentiel_solaire) AS potentiel_solaire,
         COUNT(*) AS count_etablissements,
         SUM(CASE WHEN protection THEN 1 ELSE 0 END) AS count_etablissements_proteges
    FROM etablissements
    GROUP BY code_commune
) AS agg
WHERE
  communes.code_commune = agg.code_commune
  AND communes.code_departement = '{code_departement}'
"""

conn.execute(update_query)

query = f"""
SELECT
    *,
    ST_AsGeoJSON(geom) as geometry
FROM
    communes
WHERE
    code_departement = '{code_departement}'
         """

stats_cities = conn.query(query).df()

assert stats_cities.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des communes"

stats_cities.head(3)

0    {"type":"Polygon","coordinates":[[[2.389324795...
1    {"type":"Polygon","coordinates":[[[2.507176188...
2    {"type":"Polygon","coordinates":[[[2.415272860...
Name: geometry, dtype: object

#### Mise à jour des données du Département en Base de données

In [147]:
update_query = f"""
UPDATE departements
SET 
    surface_utile = agg.surface_utile,
    potentiel_solaire = agg.potentiel_solaire,
    count_etablissements = agg.count_etablissements,
    count_etablissements_proteges = agg.count_etablissements_proteges
FROM (
    SELECT 
         code_departement,
         SUM(surface_utile) AS surface_utile,
         SUM(potentiel_solaire) AS potentiel_solaire,
         COUNT(*) AS count_etablissements,
         SUM(CASE WHEN protection THEN 1 ELSE 0 END) AS count_etablissements_proteges
    FROM etablissements
    GROUP BY code_departement
) AS agg
WHERE
    departements.code_departement = agg.code_departement
    AND departements.code_departement = '{code_departement}'
"""

conn.execute(update_query)

query = f"""
SELECT
    *,
    ST_AsGeoJSON(geom) as geometry 
FROM
    departements WHERE code_departement = '{code_departement}'
    """

stats_department = conn.query(query).df()

assert stats_department['surface_utile'].shape[0] > 0, f"Problème lors de la mise à jour du département {code_departement}"

stats_department.head(3)

,code_departement,libelle_departement,code_region,libelle_region,surface_utile,potentiel_solaire,count_etablissements,count_etablissements_proteges,geom,geometry
0,093,Seine-Saint-Denis,11,Île-de-France,867313,99163248,1431,134,"[2, 4, 0, 0, 0, 0, 0, 0, 254, 115, 18, 64, 159...","{""type"":""Polygon"",""coordinates"":[[[2.319887543..."


### Export sous forme de fichiers GeoJSON

In [ ]:
output_folder = ALGORITHME_FOLDER.parent / "results" / f"D{code_departement}"
output_folder.mkdir(exist_ok=True, parents=True)

export_to_geojson(stats_schools, output_folder / f"D{code_departement}_ecoles.geojson")
export_to_geojson(stats_cities, output_folder / f"D{code_departement}_communes.geojson")
export_to_geojson(stats_department, output_folder / f"D{code_departement}_departement.geojson")


c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


True